In [5]:
import bhavcopy
import requests
import pandas as pd
import zipfile
from datetime import datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay
import mysql.connector
import os
import numpy as np


In [1]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

if __name__ == '__main__':
    symbols = ['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'TCS.NS', 'HINDUNILVR.NS', 'KOTAKBANK.NS', 'ICICIBANK.NS', 'LT.NS', 'SBIN.NS',
               'ASIANPAINT.NS', 'AXISBANK.NS', 'MARUTI.NS', 'ITC.NS', 'NTPC.NS', 'ONGC.NS', 'HCLTECH.NS', 'M&M.NS', 'HDFCLIFE.NS', 'WIPRO.NS',
               'ULTRACEMCO.NS', 'SUNPHARMA.NS', 'HEROMOTOCO.NS', 'GRASIM.NS', 'SHREECEM.NS', 'TECHM.NS', 'COALINDIA.NS', 'BAJAJFINSV.NS',
               'NESTLEIND.NS', 'HINDALCO.NS', 'UPL.NS', 'IOC.NS', 'TITAN.NS', 'BAJFINANCE.NS', 'JSWSTEEL.NS', 'BPCL.NS', 'GAIL.NS', 'MCDOWELL-N.NS',
               'CIPLA.NS', 'HINDPETRO.NS', 'HINDZINC.NS', 'EICHERMOT.NS', 'DLF.NS', 'ADANIPORTS.NS', 'POWERGRID.NS', 'NTPC.NS', 'BRITANNIA.NS',
               'VEDL.NS']

    final_list = {'Symbol': [], 'Hammer': [], 'Inverted_Hammer': [], 'Shooting_Star': [], 'Hanging_Man': [],
                  'Bullish_Engulfing': [], 'Bearish_Engulfing': [], 'Piercing': [], 'Dark_Cloud_Cover': [],
                  'Bullish_Harami': [], 'Bearish_Harami': [], 'Bullish_kicker': [], 'Bearish_kicker': [],
                  'Morning_star': [], 'Evening_star': []}

    for i, symbol in enumerate(symbols):
        try:
            # Download historical data using yfinance's history method with date range
            data = yf.Ticker(symbol).history(period="10d")

            # Extract OHLC data for the last three days
            open1, high1, low1, close1 = data.iloc[-1][['Open', 'High', 'Low', 'Close']]
            open2, high2, low2, close2 = data.iloc[-2][['Open', 'High', 'Low', 'Close']]
            open3, high3, low3, close3 = data.iloc[-3][['Open', 'High', 'Low', 'Close']]

            # Calculate maximum and minimum values
            maximum = data['High'].max()
            minimum = data['Low'].min()

            final_list['Symbol'].append(symbol)

            # Hammer
            if (low1 <= minimum) and (abs(open1 - close1) <= 0.25 * (high1 - low1)) and (3 * (high1 - max(open1, close1)) <= (min(open1, close1) - low1)):
                final_list['Hammer'].append(1)
            else:
                final_list['Hammer'].append(0)

            # Inverted Hammer
            if (low1 <= minimum) and (abs(open1 - close1) <= 0.25 * (high1 - low1)) and ((high1 - max(open1, close1)) >= 3 * (min(open1, close1) - low1)):
                final_list['Inverted_Hammer'].append(1)
            else:
                final_list['Inverted_Hammer'].append(0)

            # Shooting Star
            if (high1 >= maximum) and (abs(open1 - close1) <= 0.25 * (high1 - close1)) and (3 * (high1 - max(open1, close1)) <= (min(open1, close1) - low1)):
                final_list['Shooting_Star'].append(1)
            else:
                final_list['Shooting_Star'].append(0)

            # Hanging Man
            if (high1 >= maximum) and (abs(open1 - close1) <= 0.25 * (high1 - close1)) and (3 * (high1 - max(open1, close1)) <= (min(open1, close1) - low1)):
                final_list['Hanging_Man'].append(1)
            else:
                final_list['Hanging_Man'].append(0)

            # Bullish Engulfing
            if (min(low1, low2) <= minimum) and (close2 < open2) and (close1 > open1) and (open1 < close2) and (close1 > open2):
                final_list['Bullish_Engulfing'].append(1)
            else:
                final_list['Bullish_Engulfing'].append(0)

            # Piercing
            if (min(low1, low2) <= minimum) and (close2 < open2) and (close1 > open1) and (open1 < close2) and (close1 >= ((open2 + close2) / 2)):
                final_list['Piercing'].append(1)
            else:
                final_list['Piercing'].append(0)

            # Bearish Engulfing
            if (max(high1, high2) >= maximum) and (close2 > open2) and (close1 < open1) and (open1 > close2) and (close1 < open2):
                final_list['Bearish_Engulfing'].append(1)
            else:
                final_list['Bearish_Engulfing'].append(0)

            # Dark Cloud Cover
            if (max(high1, high2) >= maximum) and (close2 > open2) and (close1 < open1) and (open1 > close2) and (close1 <= ((open2 + close2) / 2)):
                final_list['Dark_Cloud_Cover'].append(1)
            else:
                final_list['Dark_Cloud_Cover'].append(0)

            # Bullish Harami
            if (min(low1, low2) <= minimum) and (close2 < open2) and (close1 > open1) and (open1 > close2) and (close1 < open2):
                final_list['Bullish_Harami'].append(1)
            else:
                final_list['Bullish_Harami'].append(0)

            # Bearish Harami
            if (max(high1, high2) >= maximum) and (close2 > open2) and (close1 < open1) and (open1 < close2) and (close2 > open2):
                final_list['Bearish_Harami'].append(1)
            else:
                final_list['Bearish_Harami'].append(0)

            # Bullish Kicker
            if (min(low1, low2) <= minimum) and (close2 < open2) and (close1 > open1) and (open1 > close2) and (close1 > open2):
                final_list['Bullish_kicker'].append(1)
            else:
                final_list['Bullish_kicker'].append(0)

            # Bearish Kicker
            if (max(high1, high2) >= maximum) and (close2 > open2) and (close1 < open1) and (open1 < close2) and (close1 < open2):
                final_list['Bearish_kicker'].append(1)
            else:
                final_list['Bearish_kicker'].append(0)

            # Evening Star
            if (max(high1, high2) >= maximum) and (close3 > open3) and (open2 > close3) and (close2 > open2) and (open1 < close2) and (close1 < open3):
                final_list['Evening_star'].append(1)
            else:
                final_list['Evening_star'].append(0)

            # Morning Star
            if (min(low1, low2) <= minimum) and (close3 < open3) and (open2 < close3) and (close2 < open2) and (open1 > close2) and (close1 > open3):
                final_list['Morning_star'].append(1)
            else:
                final_list['Morning_star'].append(0)

        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    final_df = pd.DataFrame(final_list)
    final_df


In [2]:
final_df

,Symbol,Hammer,Inverted_Hammer,Shooting_Star,Hanging_Man,Bullish_Engulfing,Bearish_Engulfing,Piercing,Dark_Cloud_Cover,Bullish_Harami,Bearish_Harami,Bullish_kicker,Bearish_kicker,Morning_star,Evening_star
0,RELIANCE.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HDFCBANK.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,INFY.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TCS.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HINDUNILVR.NS,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,KOTAKBANK.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,ICICIBANK.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,LT.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,SBIN.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,ASIANPAINT.NS,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
bullish_harami_stocks = final_df[final_df['Bearish_Engulfing'] == 1]['Symbol'].tolist()
print(bullish_harami_stocks)


['NESTLEIND.NS', 'IOC.NS', 'BRITANNIA.NS']


In [22]:
df=yf.download('HDFCBANK.NS')
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1996-01-01,3.030000,3.030000,2.925000,2.980000,2.362860,350000
1996-01-02,2.980000,3.025000,2.950000,2.975000,2.358896,412000
1996-01-03,2.975000,2.995000,2.950000,2.985000,2.366825,284000
1996-01-04,2.985000,2.980000,2.940000,2.965000,2.350967,282000
1996-01-05,2.965000,2.980000,2.950000,2.960000,2.347002,189000
...,...,...,...,...,...,...
2023-12-04,1588.000000,1612.000000,1573.650024,1609.400024,1609.400024,25331228
2023-12-05,1636.500000,1639.000000,1616.500000,1623.699951,1623.699951,26115410
2023-12-06,1638.449951,1638.449951,1616.650024,1627.800049,1627.800049,19306530
